In [5]:
import numpy as np
import pandas as pd
from helpers_trans import Fy, imgs_input_fn,  multivariate_normal, find_closest_element, predict_density, bring_df_to_correct_format
import helpers_trans as hlp
import scipy.stats
from scipy.stats import norm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

In [6]:
# p = number of betas
p = 10

In [7]:
extracted_coefficients_directory = '../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
y_path = str(extracted_coefficients_directory + 'Bzeta/labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
true_z = np.load(z_path)
true_y = np.load(y_path)

# filtered Ridge 
va_ridge_dir = '../../data/commaai/va/filtered_gaussian_resampled/Ridge/'
mu_t_va = np.genfromtxt(va_ridge_dir + 'mu_t_va.csv', delimiter = ',')
beta = np.mean(mu_t_va[int(0.9*50000):50000,0:10], axis = 0)
tau_sq = np.exp(np.mean(mu_t_va[int(0.9*50000):50000,10], axis = 0))
#vartheta_path = '../../data/density/03082020/vartheta_final.csv'
#B_zeta_path = '../../bdd100k_test_data/extracted_coefficients/10092020/B_zeta_predictions_val.csv'
#z_path = '../../data/tfrecords/03082020/val_yaw_transformed.csv'
density_path = '../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)
#true_y_path = '../../data/tfrecords/03082020/val_yaw_original.csv'


#vartheta = np.genfromtxt(vartheta_path, delimiter=',')
#B_zeta = np.genfromtxt(B_zeta_path, delimiter=',')
n = B_zeta.shape[0]

In [ ]:
# get basis function for new observation

In [4]:
grid = np.linspace(min(density['axes']), max(density['axes']), 10000)
density_y = density['axes']
density_pdf = density['pdf']
# compute these beforehand to save computation time
p_y_y0 = [density_pdf[find_closest_element(y_i,density_y)] for y_i in grid]
part_1 = np.array([norm.ppf(Fy(y_i, density)) for y_i in grid])
phi_1_z = np.array([scipy.stats.norm(0, 1).pdf(y_i) for y_i in part_1])

In [ ]:
def predict_density(x, grid, p_y_y0, part_1, phi_1_z, beta, tau_sq):
    
    psi_x0 = x
    
    f_eta_x0 = psi_x0.dot(beta)
    s_0_hat = (1 + tau_sq*psi_x0.dot(psi_x0))**(-0.5)
    
    part_0 = s_0_hat*f_eta_x0

    # compute the cdf of new ys
    
    term_1 = scipy.stats.norm(0, 1).pdf((part_1- part_0) / s_0_hat)

    p_y_single_obs_whole_dens = (p_y_y0/phi_1_z)*(1/s_0_hat)*term_1
    
    return(p_y_single_obs_whole_dens)